# Занятие № 1 по клиентской аналитике

## выполнил Колеганов Николай

# Задание:
1. Залить в свою БД данные по продажам (часть таблицы Orders в csv, исходник здесь
https://drive.google.com/drive/folders/1C3HqIJcABblKM2tz8vPGiXTFT7MisrML?usp=sharin
g )
2. Проанализировать, какой период данных выгружен
3. Посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, кот
совершали заказы.
4. По годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать
вывод , как изменялись это показатели Год от года.
5. Найти кол-во пользователей, кот покупали в одном году и перестали покупать в
следующем.
6. Найти ID самого активного по кол-ву покупок пользователя.

## Загрузка библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Загружаем БД по продажам

In [2]:
file = files.upload()

Saving orders_20190822.csv to orders_20190822.csv


In [3]:
!ls

orders_20190822.csv  sample_data


In [4]:
orders = pd.read_csv('orders_20190822.csv', sep=';')
a = orders.shape[0]
print(f'Первоначальное количество строк = {a}')

Первоначальное количество строк = 2002804


In [5]:
orders.head()

,id_o,user_id,price,o_date
0,1234491,337544,"539,000",01.01.2016
1,1234494,171642,"153,300",01.01.2016
2,1234497,260596,"55,300",01.01.2016
3,1234498,1105609,"752,500",01.01.2016
4,1234500,982696,"4410,000",01.01.2016


## конвертация данных

In [0]:
def convert_data(data):
    new_data = data.replace(',','.')
    return float(new_data)

In [0]:
orders['price'] = orders['price'].apply(convert_data)

In [0]:
orders['o_date'] = pd.to_datetime(pd.Series(orders['o_date']), format="%d.%m.%Y")

In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002804 entries, 0 to 2002803
Data columns (total 4 columns):
 #   Column   Dtype         
---  ------   -----         
 0   id_o     int64         
 1   user_id  int64         
 2   price    float64       
 3   o_date   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 61.1 MB


## убираем строки с неположительной и слишком большой ценой

In [0]:
negative_list = orders.loc[(orders['price'] <= 0)|(orders['price'] > 100000)]
orders = orders.drop(negative_list.index, axis = 0)

In [11]:
orders.head()

,id_o,user_id,price,o_date
0,1234491,337544,539.0,2016-01-01
1,1234494,171642,153.3,2016-01-01
2,1234497,260596,55.3,2016-01-01
3,1234498,1105609,752.5,2016-01-01
4,1234500,982696,4410.0,2016-01-01


## 1. проанализируем, какой период данных выгружен

In [13]:
print('Период времени до ',orders['o_date'].max())
print('Период времени от ', orders['o_date'].min())

Период времени до  2017-12-31 00:00:00
Период времени от  2016-01-01 00:00:00


## 2. посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, которые совершали заказы.

In [14]:
b = orders.shape[0]
print(f'Количество строк после удаления некорректных = {b}')

Количество строк после удаления некорректных = 2002696


In [15]:
print(f'После того, как были убраны строки с неположительной и слишком большой ценой количество строк сократилось на {a - b}')

После того, как были убраны строки с неположительной и слишком большой ценой количество строк сократилось на 108


In [16]:
number_of_orders = orders['id_o'].unique()
len(number_of_orders)

2002696

In [17]:
number_of_users = orders['user_id'].unique()
len(number_of_users)

1015088

## 3. по годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать вывод , как изменялись это показатели год от года.

In [18]:
mean_order = orders.groupby(orders['o_date'].dt.year, as_index=False)[['price']].mean().rename(columns={'price':'mean_price'})
mean_order

,mean_price
0,2090.294426
1,2392.783860


In [0]:
users_2016 = orders.loc[orders['o_date'].dt.year == 2016, 'user_id'].values
unique_users_2016 = orders.loc[orders['o_date'].dt.year == 2016, 'user_id'].unique()
order_per_user_2016 = len(users_2016) / len(unique_users_2016)

In [20]:
print('Результат за 2016 г. - ', order_per_user_2016)

Результат за 2016 г. -  1.935104238984639


In [0]:
users_2017 = orders.loc[orders['o_date'].dt.year == 2017, 'user_id'].values
unique_users_2017 = orders.loc[orders['o_date'].dt.year == 2017, 'user_id'].unique()
order_per_user_2017 = len(users_2017) / len(unique_users_2017)

In [22]:
print('Результат за 2017 г. - ', order_per_user_2017)

Результат за 2017 г. -  1.742957402362599


# Вывод
## средний чек вырос с 2090 руб. в 2016 г. до 2393 руб. в 2017 г.
## среднее количество заказов на покупателя упало с 1.93 в 2016 г. до 1.74 в 2017 г.

## 4. найти кол-во пользователей, которые покупали в одном году и перестали покупать в следующем.

In [23]:
ex_users = []
for i in unique_users_2016:
    if i not in unique_users_2017:
        ex_users.append(i)
len(ex_users)

360216

## 5. найти ID самого активного по кол-ву покупок пользователя.

так как данных очень много (более 2 млн. строк) выполнение кода может занять очень большое количество времени, а так же значительные вычислительные ресурсы компьютера, поэтому решил сделать выборку в 600000 первых строк БД, что составляет примерно 30 % от всей БД

In [0]:
orders = orders[0:600000]

In [26]:
max_value = 1
max_user_id = 1

for i in orders['user_id']:
    if (orders['user_id'] == i).sum() > max_value:
        max_value = (orders['user_id'] == i).sum()
        max_user_id = i
        a = orders[orders['user_id'] == i].index
        orders['user_id'].drop(a)
    
max_value, max_user_id

(1210, 765861)

In [27]:
print(f'На выборке из первых 600000 строк победил user #{max_user_id}, который произвёл {max_value} покупок.')

На выборке из первых 600000 строк победил user #765861, который произвёл 1210 покупок.
